In [2]:
import json
train_conversations_path = './data/train_set/train_set_ALL.json'
PREFIX_E = 'http://www.wikidata.org/entity/'

# load training set
with open(train_conversations_path, "r") as data:
    conversations = json.load(data)
print("%d conversations loaded"%len(conversations))

6720 conversations loaded


In [3]:
# load KG
from hdt import HDTDocument, TripleComponentRole
from settings import *

hdt_file = 'wikidata20200309.hdt'
kg = HDTDocument(hdt_path+hdt_file)

max_triples = 500000000
offset = 0
rdfsLabelURI='http://www.w3.org/2000/01/rdf-schema#label'


# load adjacencies
def generate_adj_sp(adjacencies, n_entities, include_inverse):
    '''
    Build adjacency matrix
    '''
    adj_shape = (n_entities, n_entities)
    
    # colect all predicate matrices separately into a list
    sp_adjacencies = []
    for edges in adjacencies:
        # split subject (row) and object (col) node URIs
        n_edges = len(edges)
        row, col = np.transpose(edges)
        
        # duplicate edges in the opposite direction
        if include_inverse:
            _row = np.hstack([row, col])
            col = np.hstack([col, row])
            row = _row
            n_edges *= 2
        
        # create adjacency matrix for this predicate
        data = np.ones(n_edges)
        adj = sp.csr_matrix((data, (row, col)), shape=adj_shape)
        sp_adjacencies.append(adj)
    
    return np.asarray(sp_adjacencies)


def lookup_entity_labels(e_uri):
    # prepare mapping tensors with entity labels and ids
    (triples, cardinality) = kg.search_triples(e_uri, rdfsLabelURI, "")
    if cardinality > 0:
        while True:
            try:
                label = triples.next()[2]
                # "(They Long to Be) Close to You"@en
                label = label.split('"@')
                if label[-1] == 'en':
                    # strip language marker
                    return label[0][1:]
            except:
                return None

In [4]:
# load a sample conversation as a sequence of entities
import numpy as np
import scipy.sparse as sp
from collections import Counter

entity_answers = []
subgraph_entities = []
subgraph_relations = []
overlaps = []
disconnected = []

for j, conversation in enumerate(conversations[:]):
    entities = []
    seed_entity = conversation['seed_entity']
    entities.append(PREFIX_E + seed_entity.split('/')[-1])
    n_questions = len(conversation['questions'])
    
    questions = []
    answer_entities = []
    answer_labels = []
    for i in range(n_questions):
        question = conversation['questions'][i]['question']
        questions.append(question)
        
        answers = conversation['questions'][i]['answer'].split(';')
        
        _answer_entities = []
        _answer_labels = []
        for answer in answers:
            # consider only answers which are entities
            if ('www.wikidata.org' in answer): 
                entity = PREFIX_E + answer.split('/')[-1]
                entities.append(entity)
                _answer_entities.append(entity)
                # look up entity label
                entity_label = lookup_entity_labels(entity)
                print(entity_label)
                _answer_labels.append(entity_label)
            else:
                _answer_entities.append(None)
                _answer_labels.append(answer)
        
        answer_entities.append(_answer_entities)
        answer_labels.append(_answer_labels)
    
    n_answers = len(entities)-1
    entity_answers.append(n_answers)
    
    # look up entity ids in the KG: global ids
    matched_entity_ids = []
    for entity in entities:
        global_id = kg.string_to_global_id(entity, TripleComponentRole.OBJECT)
        matched_entity_ids.append(global_id)

    # retrieve relevant subgraph
    subgraph = kg.compute_hops(matched_entity_ids, max_triples, offset)
    entity_ids, predicate_ids, adjacencies = subgraph
    
    # dump sample with subgraph as json
    data = {'seed_entity': seed_entity, 'questions': questions,
            'answer_entities': answer_entities, 'answer_labels': answer_labels,
            'entities': entity_ids, 'predicates': predicate_ids, 'adjacencies': adjacencies}
    json_object = json.dumps(data)
    with open('./data/subgraphs/train_set/%d.json'%j, "w") as outfile:
        outfile.write(json_object) 
#     break

(They Long to Be) Close to You
anorexia nervosa
heart failure
Danny Elfman
(They Long to Be) Close to You
anorexia nervosa
heart failure
Haruki Murakami
Japan
Kafka on the Shore
J. K. Rowling
Harry Potter and the Philosopher's Stone
Harry Potter and the Deathly Hallows
Hermione Granger
Fox Broadcasting Company
Los Angeles
Martin Riggs
Roger Murtaugh
Clayne Crawford
Damon Wayans
United Kingdom
The Beatles
The Who
London
Pink Floyd
Roger Waters
Syd Barrett
David Gilmour
The Wall
David Crane
Marta Kauffman
Warner Bros. Domestic Television Distribution
I'll Be There for You
Friends
Graceland
United States Army
Gelsenkirchen
North Rhine-Westphalia
Düsseldorf
Fortuna Düsseldorf
Chadwick Boseman
Ryan Coogler
Los Angeles
hip hop music
techno
pop music
Ken Kesey
Oregon
Houses of the Holy
Jimmy Page
The Yardbirds
Haruki Murakami
Japan
Kafka on the Shore
Manchester United F.C.
Victoria Beckham
Spice Girls
Best MLS Player ESPY Award
United Kingdom
J. K. Rowling
Harry Potter and the Chamber of Secr

Mikhail Bulgakov
Mikhail Lermontov
Daniel Levy
United Kingdom
London
White Hart Lane
NBC
Lorne Michaels
Toronto
George Carlin
The Great Gatsby
F. Scott Fitzgerald
Zelda Fitzgerald
Frances Scott Fitzgerald
Breaking Bad
Better Call Saul
E. B. White
Araneae
pig
children's novel
fairy tale
SV Auersmacher
1. FC Köln
Saarbrücken
forward
winger
Brazil national football team
FIFA Player of the Century
Santos F.C.
New York Cosmos
Eddie Murphy
Coming to America
Mark Twain
Mississippi River
Chatto & Windus
Huckleberry Finn
Chadwick Boseman
Ryan Coogler
Friends
Neil Patrick Harris
Martin Scorsese
The Wolf of Wall Street
The Wolf of Wall Street
mockumentary
American television sitcom
Los Angeles
North America
Richard Adams
English
country music
Southeastern Oklahoma State University
Pittsburg County
Stephen King
post-apocalyptic novel
horror novel
Doubleday
Houses of the Holy
Jimmy Page
The Yardbirds
Orson Scott Card
science fiction
Howard Deutch
John Hughes
Dan Aykroyd
John Candy
Universal Studios

2010 FIFA World Cup
2014 FIFA World Cup
BBVA Stadium
Major League Soccer
Wilmer Cabrera
Colombia
science fiction
fantasy
Atlanta
Jackson
Netflix
David Lynch
Tim Hunter
Elena Gilbert
Katherine Pierce
Amara
Katerina Graham
Tom Cruise
Christopher McQuarrie
thriller film
Bad Robot Productions
Gazprom
Spartak Moscow
Otkrytie Arena
hip hop music
Kim Kardashian
The College Dropout
Chicago State University
René Angélil
cancer
My Heart Will Go On
Unison
drama
Kevin Costner
Taylor Sheridan
Paramount Network
Montana
Spain
Seville
David Crane
Marta Kauffman
Warner Bros. Domestic Television Distribution
I'll Be There for You
Friends
Manchester United F.C.
Victoria Beckham
Spice Girls
Best MLS Player ESPY Award
United Kingdom
CBS
Loretta Swit
Suicide Is Painless
Benjamin Franklin 'Hawkeye' Pierce
SV Auersmacher
1. FC Köln
Saarbrücken
Chadwick Boseman
Michael B. Jordan
Human Torch
Ryan Coogler
The Blacklist, season 3
James Spader
Leslie Stefanson
Charles Dickens
Chapman & Hall
black comedy
London
Ken

heart failure
Major League Soccer
Dignity Health Sports Park
Cobi Jones
Steve Franks
USA Network
James Roday
Shawn Spencer
Santa Barbara
Tim Allen
Tom Hanks
Buzz Aldrin
Buzz Lightyear of Star Command
Disney Television Animation
Pixar
Natalie Portman
Keira Knightley
Sabé
Breakfast at Tiffany's
Blake Edwards
Truman Capote
Richard Adams
English
Charles Dickens
Chapman & Hall
black comedy
London
Kent
Howard Deutch
John Hughes
Dan Aykroyd
John Candy
Universal Studios
United States of America
Canada
Santos F.C.
New York Cosmos
Larry Gelbart
CBS
Robert Altman
Korean War
Alan Alda
Richard Adams
England
Newbury
Berkshire
Danny Elfman
Mia Farrow
America
folk rock
Haruki Murakami
Japan
Kafka on the Shore
American football
Michael Oher
John Lee Hancock
The Blind Side: Evolution of a Game
Madonna
dance-pop
Nile Rodgers
Sire
Friends
United States of America
United Kingdom
Los Angeles
hip hop music
techno
pop music
J. K. Rowling
Harry Potter and the Chamber of Secrets
Harry Potter and the Prisoner of

Danny Elfman
midfielder
Premier League
Chingford Foundation School
Victoria Beckham
Chadwick Boseman
Michael B. Jordan
Human Torch
Ryan Coogler
United States of America
D. B. Woodside
Tom Ellis
Stephen King
United States of America
Canada
D.C. United
Foxboro Stadium
American football
Michael Oher
John Lee Hancock
The Blind Side: Evolution of a Game
George Lucas
Star Wars Episode IV: A New Hope
Mark Hamill
Carrie Fisher
Daniel Levy
United Kingdom
London
White Hart Lane
Manchester United F.C.
United Kingdom
Russia
Julia Garner
Charlie Cox
Finland
Finnish
Swedish
English
Finland
Thom Yorke
Philip Selway
Pablo Honey
Chadwick Boseman
Ryan Coogler
Madonna
dance-pop
Nile Rodgers
Sire
None
Markus Heitz
Homburg
Friends
United States of America
United Kingdom
The CW
Elena Gilbert
Nina Dobrev
Russia
Qatar
Mr. Mercedes
Portland
James Cameron
Pandora
James Horner
Jake Sully
Neytiri
Donald Glover
Because the Internet
STN MTN / Kauai
Breakfast at Tiffany's
Blake Edwards
Truman Capote
It
The Dark Towe

...

José Mourinho
Los Angeles
Mickey Madden
Austin
Morticia Addams
Christina Ricci
Jimmy Workman
Christina Ricci
The Fellowship of the Ring
Bloemfontein
Edith Tolkien
France
Real Sociedad
Atlético Madrid
Diego Simeone
England
John Bonham
Led Zeppelin
Robert Plant
Wales national football team
Gareth Bale
UEFA Euro 2016
Portugal national football team
Brad Bird
Samuel L. Jackson
Elena Gilbert
Katherine Pierce
Amara
Katerina Graham
Mr. Mercedes
Portland
United Kingdom
The Beatles
The Who
London
CBS
Loretta Swit
Suicide Is Painless
Benjamin Franklin 'Hawkeye' Pierce
Sevier County
Carl Thomas Dean
Rebecca
9 to 5
Michael Wolff
Donald Trump
Spain
Chelsea F.C.
Chelsea F.C.
Spain national football team
Douglas Adams
Eoin Colfer
Artemis Fowl
Sandra Bullock
Netflix
Susanne Bier
R
Miroslav Klose
Argentina national football team
Poland
Edith Tolkien
John Francis Reuel Tolkien
Breakfast at Tiffany's
Blake Edwards
Truman Capote
Harper Lee
Southern Gothic
Bildungsroman
To Kill a Mockingbird
Robert Mulliga

Martin Scorsese
The Wolf of Wall Street
The Wolf of Wall Street
Mr. Mercedes
Portland
Spain
Seville
Friends
United States of America
United Kingdom
Denzel Washington
Columbia Pictures
Boston
Harry Gregson-Williams
Chloë Grace Moretz
Brian Schmetzer
Major League Soccer
Garth Lagerwey
Martin Scorsese
The Wolf of Wall Street
The Wolf of Wall Street
Rick Sanchez
scientist
adult
G
Digital Fortress
Seville
National Security Agency
Jared Leto
America
30 Seconds to Mars
Tobias Schweinsteiger
association football player
Julie Plec
Kevin Williamson
Nina Dobrev
Paul Wesley
Ian Somerhalder
Elena Gilbert
Katherine Pierce
Amara
D. B. Woodside
Tom Ellis
Brian Schmetzer
Major League Soccer
Garth Lagerwey
Inception
Christopher Nolan
Gelsenkirchen
North Rhine-Westphalia
Düsseldorf
Fortuna Düsseldorf
Daniel Levy
United Kingdom
London
White Hart Lane
Real Salt Lake
fullback
Greensboro
Lisa Marie Presley
To Whom It May Concern
E. B. White
Araneae
pig
children's novel
fairy tale
Zac Efron
Troy Bolton
Vaness

Dresden
Elena Gilbert
Katherine Pierce
Amara
Katerina Graham
The Great Gatsby
F. Scott Fitzgerald
Zelda Fitzgerald
Frances Scott Fitzgerald
NBC
Lorne Michaels
Toronto
George Carlin
Jonny Lee Miller
Lucy Liu
canon of Sherlock Holmes
CBS
David Shore
Gregory House
medical drama
Chris Martin
The Chainsmokers
Coldplay
Memories...Do Not Open
Real Salt Lake
fullback
Greensboro
Thom Yorke
In Rainbows
The King of Limbs
Spain
Seville
FC Bayern Munich
Ottmar Hitzfeld
Peter Benchley
Doubleday
Bantam Books
thriller
United States of America
Wisconsin
Mark Brazill
Steve Franks
USA Network
James Roday
Shawn Spencer
Santa Barbara
L. Frank Baum
The Wizard of Oz
Glinda the Good Witch
children's literature
Eddie Redmayne
Crazy Rich Asians
Jon M. Chu
Adele Lim
Denzel Washington
Columbia Pictures
Boston
Harry Gregson-Williams
Chloë Grace Moretz
David Shore
Gregory House
medical drama
Friends
Jennifer Aniston
Ross Geller
Monica Geller
Zac Efron
Troy Bolton
Vanessa Hudgens
Ashley Tisdale
Kenny Ortega
Premier 

Ken Kesey
Oregon
Californication
John Frusciante
Trickfinger II
David Lynch
Tim Hunter
American football
Michael Oher
John Lee Hancock
The Blind Side: Evolution of a Game
Eric Kripke
Jeffrey Dean Morgan
Azazel
Jim Beaver
Stephen King
post-apocalyptic novel
horror novel
Doubleday
Iron Man 3
Shane Black
Pittsburgh
Pennsylvania
Los Angeles
Mickey Madden
Austin
Bruce Willis
John McClane
John McTiernan
Nothing Lasts Forever
Real Salt Lake
fullback
Greensboro
Patricia Arquette
Phoenix
Miguel Sandoval
Jake Weber
Howard Deutch
John Hughes
Dan Aykroyd
John Candy
Universal Studios
Claire Foy
Liverpool John Moores University
Oxford School of Drama
Matt Smith
Elizabeth II
Peter Morgan
Juventus F.C.
Italy
France national football team
Danny Elfman
Sevier County
Carl Thomas Dean
Rebecca
9 to 5
Manchester United F.C.
Victoria Beckham
Spice Girls
Best MLS Player ESPY Award
United Kingdom
FC Barcelona
Lionel Messi
Argentina
Spain
Matt Groening
Homer Simpson
Dan Castellaneta
Marge Simpson
Alex Garland
L

Manchester United F.C.
Real Madrid CF
Paris Saint-Germain F.C.
Victoria Beckham
science fiction
fantasy
Atlanta
Jackson
Netflix
Manchester United F.C.
United Kingdom
Russia
FC Barcelona
Lionel Messi
Argentina
Spain
Steve Franks
USA Network
James Roday
Shawn Spencer
Santa Barbara
Wes Craven
Neve Campbell
Kevin Williamson
Gale Weathers
United States of America
UEFA Super Cup
FIFA Club World Cup
Real Madrid CF
Liverpool F.C.
Gareth Bale
Charlie Sheen
The Execution of Private Slovik
Jane Austen
Elizabeth Bennet
Mr. Darcy
Mansfield Park
Friends
United States of America
United Kingdom
Wisconsin
Mark Brazill
Santos F.C.
New York Cosmos
Los Angeles
Mickey Madden
Austin
SV Auersmacher
1. FC Köln
Saarbrücken
Isaac Asimov
Foundation and Empire
Morticia Addams
Christina Ricci
Jimmy Workman
Christina Ricci
Martin Scorsese
The Wolf of Wall Street
The Wolf of Wall Street
Breaking Bad
Better Call Saul
Manchester United F.C.
Victoria Beckham
Spice Girls
Best MLS Player ESPY Award
United Kingdom
E. B. W

In [5]:
def stats(_list):
    print(min(_list), np.mean(_list), max(_list))
    print(Counter(_list))
    print('\n')

stats(entity_answers)

0 3.4285714285714284 8
Counter({4: 2368, 3: 1696, 2: 1184, 5: 864, 1: 352, 6: 160, 7: 32, 8: 32, 0: 32})


